# Assignment1 "Creating Visualization"

## Prerequisits
Installing the Python Library Pandas and Numpy

In [1]:
%pip install pandas
%pip install numpy
%pip install wandb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install Matplotlib
%pip install Seaborn
%pip install Plotly

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import wandb

In [4]:
wandb.init(project="emotion-bias-project")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: elsaesserniklas. Use `wandb login --relogin` to force relogin


In [10]:
import glob
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

# List to store prediction results and image paths
predictions = []

# Load your face recognition model
face_emotion_model_path = '/Users/niklaselsasser/Code/DataVisualization/emotion_model_fv_e10.h5'
face_recognition_model = load_model(face_emotion_model_path)

# Function to preprocess images
def preprocess_image(img):
    img = cv2.resize(img, (48, 48))
    img = img.astype('float') / 255.0  # Normalize pixel values
    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    return img

# File path to the uploaded image
#uploaded_img_path = '/content/drive/MyDrive/Faces/testing/niklas_vhappy.jpg'
uploaded_img_folder_path = '/content/drive/MyDrive/Faces/testing'

# Read the uploaded image
allFiles = glob.glob(glob.escape(uploaded_img_folder_path) + "/*.jpg")

for imgPath in allFiles:
    img = cv2.imread(imgPath, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print("Error: Unable to read the image.")
    else:
        # Preprocess the image
        processed_img = preprocess_image(img)
        # Predict gender using the model
        predicted_gender = face_recognition_model.predict(processed_img)

        # Map the predicted index to gender and emotion labels
        genders = ["men", "women"]
        emotions = ["happy", "neutral"]

        gender_index = np.argmax(predicted_gender)
        gender_label = genders[gender_index // len(emotions)]
        emotion_label = emotions[gender_index % len(emotions)]

        # Extract only the filename from the full image path
        img_filename = os.path.basename(imgPath)

        # TESTING SINGLE PICTURES AND SHOWING THE PICTURE
        # Print the predicted gender and emotion with the image filename
        #print(f"Image Path: " + img_filename)
        #print(f"Predicted Gender: {gender_label.capitalize()}")
        #print(f"Predicted Emotion: {emotion_label.capitalize()}")

        #faceImg = cv2.imread(imgPath)
        #plt.imshow(faceImg)
        #--------------------------------------------------

        # Append prediction results (filename only) and gender/emotion labels to the list
        predictions.append([img_filename, gender_label.capitalize(), emotion_label.capitalize()])


OSError: No file or directory found at /content/drive/MyDrive/Faces/models/emotion_model_fv_e10.h5

In [9]:
import pandas as pd

# Create a DataFrame from the predictions list
df = pd.DataFrame(predictions, columns=["Image Path", "Predicted Gender", "Predicted Emotion"])

# Save the DataFrame as a CSV file
output_csv_path = '/content/drive/MyDrive/Faces/predictions/predictions_fv_e10_1.csv'
df.to_csv(output_csv_path, index=False)

print(f"Prediction results saved at: {output_csv_path}")
print("the table/n", df)


NameError: name 'predictions' is not defined

## Additional Task
&#9745; Experimentieren Sie mit verschiedenen Python-Visualisierungsbibliotheken und deren Stärken.\
&#9745; Erstellen Sie interaktive Visualisierungen mit Bibliotheken wie Plotly.\
&#9745; Integrieren Sie Ihr Python-Skript in ein Jupyter Notebook für eine strukturierte Analyse.

### 1. Additional Python-Visualizationlibraries
Bertopic from *https://maartengr.github.io/BERTopic/getting_started/quickstart/quickstart.html*

**Bertopic**

In [5]:
%pip install bertopic

Note: you may need to restart the kernel to use updated packages.


**Altair**\
from *https://altair-viz.github.io/getting_started/overview.html#overview*

In [6]:
%pip install altair
# import altair with an abbreviated alias
import altair as alt

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install altair vega_datasets

Note: you may need to restart the kernel to use updated packages.


In [8]:
import altair as alt
from vega_datasets import data

source = data.seattle_weather()

zoom = alt.selection_interval(encodings=["x", "y"])

minimap = (
    alt.Chart(source)
    .mark_point()
    .add_params(zoom)
    .encode(
        x="date:T",
        y="temp_max:Q",
        color=alt.condition(zoom, "weather", alt.value("lightgray")),
    )
    .properties(
        width=200,
        height=200,
        title="Minimap -- click and drag to zoom in the detail view",
    )
)

detail = (
    alt.Chart(source)
    .mark_point()
    .encode(
        alt.X("date:T").scale(domain={"param": zoom.name, "encoding": "x"}),
        alt.Y("temp_max:Q").scale(domain={"param": zoom.name, "encoding": "y"}),
        color="weather",
    )
    .properties(width=600, height=400, title="Seattle weather -- detail view")
)

detail | minimap

alt.HConcatChart(...)